<a href="https://colab.research.google.com/github/EduSoBarbosa/Web-Scrapping-Hqs/blob/main/Web_Scrapping_Panini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pips

In [64]:
!pip install requests
!pip install beautifulsoup4
!pip install pyTelegramBotAPI
!pip install schedule


### Imports

In [65]:
import requests
import threading
import telebot
from telebot import types
import time
import logging
from telebot.types import ReplyKeyboardMarkup
from bs4 import BeautifulSoup
from itertools import chain


## **Extraindo url e Criando função basica**

In [66]:
url = 'https://panini.com.br/desconto-saldao?editorial=DC+Comics&product_list_order=percentage_discount'
url2 = 'https://panini.com.br/desconto-saldao?editorial=DC+Comics&p=2&product_list_order=percentage_discount'
url3 = 'https://panini.com.br/desconto-saldao?editorial=DC+Comics&p=3&product_list_order=percentage_discount'

response = requests.get(url)
response2 = requests.get(url2)
response3 = requests.get(url3)

soup = BeautifulSoup(response.text, 'html.parser')
soup2 = BeautifulSoup(response2.text, 'html.parser')
soup3 = BeautifulSoup(response3.text, 'html.parser')
#Procurar links com partes do html
#links_editoriais = soup.select('a[href*="desconto-saldao?editorial="]')
#print(f"Encontrados {len(links_editoriais)} links de editoriais em promoção.")

lista_Produtos = soup.find_all('div', class_="product-item-info")
lista_Produtos2 = soup2.find_all('div', class_="product-item-info")
lista_Produtos3 = soup3.find_all('div', class_="product-item-info")

In [67]:
print(f'Encontramos {len(lista_Produtos) + len(lista_Produtos2) + len(lista_Produtos3)} price box')

Encontramos 30 price box


In [68]:
dados_produtos = []
def Extrair():
    for produto_html in chain(lista_Produtos, lista_Produtos2, lista_Produtos3):
        tag_nome = produto_html.find('a', class_='product-item-link')
        if tag_nome:
            nome = tag_nome.text.strip()
            nome = nome.upper()
            link = tag_nome.get('href', 'x')
        else:
            continue

        tag_preco = produto_html.find('span', class_='price')
        if tag_preco:
            preco = tag_preco.text.strip()
            preco = preco.replace("\xa0", '')
        else:
            continue

        tag_preco_antigo = produto_html.find('span', class_='old-price')
        if tag_preco_antigo:
            preco_antigo = tag_preco_antigo.text.strip()
            preco_antigo = preco_antigo.replace('Preço', '')
            preco_antigo = preco_antigo.replace("\xa0", '')
        else:
            continue

        dados_produtos.append({
            'Nome': nome,
            'Preço': preco,
            'Preço Antigo': preco_antigo,
            'Link': link

    })


## Funções de Tratamento

In [69]:
class CategoriaProdutos:
    def __init__(self, nome):
        self.nome = nome
        self.tags = []

    def adicionar_tags(self, produto):
        self.tags.append(produto)

    def mostrar(self):
        texto = ''
        if not self.tags:
            texto = f"Não há produtos na categoria {self.nome}."
            return texto
        else:
          for produto in self.tags:
              texto += f"{produto['Nome']} | Preço: {produto['Preço']} | Preço Antigo: {produto['Preço Antigo']} \n{produto['Link']}\n\n"
          return texto

In [70]:
#tags
Batman = CategoriaProdutos('BATMAN')
Superman = CategoriaProdutos('SUPERMAN')
Maravilha = CategoriaProdutos('MULHER-MARAVILHA')
Lanterna_Verde = CategoriaProdutos('LANTERNA VERDE')
Constantine = CategoriaProdutos('HELLBLAZER')
LigaDaJustiça = CategoriaProdutos('LIGA DA JUSTIÇA')
Sandman = CategoriaProdutos('SANDMAN')
Lucifer = CategoriaProdutos('LÚCIFER')
Alvo_Humano = CategoriaProdutos('ALVO HUMANO')
Mulher_Gato = CategoriaProdutos('MULHER GATO')
Asa_Noturna = CategoriaProdutos('ASA NOTURNA')
Aquaman = CategoriaProdutos('AQUAMAN')

Extrair()

def l():
    mapa = {
        "BATMAN": Batman,
        "SUPERMAN": Superman,
        "MULHER MARAVILHA": Maravilha,
        "LANTERNA VERDE": Lanterna_Verde,
        "HELLBLAZER": Constantine,
        "LIGA DA JUSTIÇA": LigaDaJustiça,
        "SANDMAN": Sandman,
        "LUCIFER": Lucifer,
        "ALVO": Alvo_Humano,
        "MULHER GATO": Mulher_Gato,
        "ASA NOTURNA": Asa_Noturna,
        "AQUAMAN": Aquaman
    }

    for produto in dados_produtos:
        nome = produto["Nome"].upper()
        for chave, classe in mapa.items():
            if chave in nome:
                classe.adicionar_tags(produto)

l()


# Bot telegram

In [71]:
bot = telebot.TeleBot('7773874902:AAHvKjQ-Ifb2WtsyBiHjeYGL-lWtY4nf70s')
dados = dados_produtos
@bot.message_handler(commands=['start'])
def start(msg:telebot.types.Message): # type: ignore
  texto =  f'Olá seja bem vindo, Gostaria de ver todas as promoções ou usar tags?'
  bot.send_message(msg.chat.id, texto)
  time.sleep(1)
  texto = '''
/Todas
/Tags
  '''
  bot.reply_to(msg, texto)



@bot.message_handler(commands=['Todas'])
def Todas(msg:telebot.types.Message): # type: ignore
  mensagem = ''
  for produto in dados_produtos:
      mensagem += f"{produto['Nome']} | Preço: {produto['Preço']} | Preço Antigo:{produto['Preço Antigo']} \n {produto['Link']} \n"
  bot.send_message(msg.chat.id, mensagem)



@bot.message_handler(commands=['Tags'])
def Tags(msg:telebot.types.Message): # type: ignore
   texto = f'Abaixo a Lista de Tags Disponiveis'
   bot.send_message(msg.chat.id, texto)
   time.sleep(1)
   texto = '''
1. /Batman

2. /Superman

3. /Mulher_Maravilha

4. /Lanterna_Verde

4. /Constantine

6. /LigaDaJustica

7. /Sandman

8. /Lucifer

9. /Alvo_Humano

10. /Mulher_Gato

11. /Asa_Noturna

12. /Aquaman
'''
   bot.send_message(msg.chat.id, texto)


@bot.message_handler(commands=['Batman'])
def mostrar_batman(msg:telebot.types.Message): # type: ignore
    texto = Batman.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Superman'])
def mostrar_Superman(msg:telebot.types.Message): # type: ignore
    texto = Superman.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Mulher_Maravilha'])
def mostrar_Mulher_Maravilha(msg:telebot.types.Message): # type: ignore
    texto = Maravilha.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Lanterna_Verde'])
def mostrar_Lanterna_Verde(msg:telebot.types.Message): # type: ignore
    texto = Lanterna_Verde.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Constantine'])
def mostrar_Constantine(msg:telebot.types.Message): # type: ignore
    texto = Constantine.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['LigaDaJustica'])
def mostrar_liga(msg:telebot.types.Message): # type: ignore
    texto = LigaDaJustiça.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Sandman'])
def mostrar_Sandman(msg:telebot.types.Message): # type: ignore
    texto = Sandman.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Lucifer'])
def mostrar_Lucifer(msg:telebot.types.Message): # type: ignore
    texto = Lucifer.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Alvo_Humano'])
def mostrar_Alvo_Humano(msg:telebot.types.Message): # type: ignore
    texto = Alvo_Humano.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Mulher_Gato'])
def mostrar_Mulher_Gato(msg:telebot.types.Message): # type: ignore
    texto = Mulher_Gato.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Asa_Noturna'])
def mostrar_Asa_Noturna(msg:telebot.types.Message): # type: ignore
    texto = Asa_Noturna.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)

@bot.message_handler(commands=['Aquaman'])
def mostrar_Aquaman(msg:telebot.types.Message): # type: ignore
    texto = Aquaman.mostrar()
    bot.send_message(msg.chat.id, texto)
    time.sleep(1)
    texto = '/start'
    bot.send_message(msg.chat.id, texto)





bot.infinity_polling()

2025-09-28 03:07:40,287 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-09-28 03:07:40,291 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
